In [1]:
import os, pickle, sys, torch, numpy as np

# paths  
# table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/all_2d1d_datasets/entropy_radius_exps_on_HSQC'

print_keys = [
    "test/mean_rank_1",
    "test/mean_rank_5",
    "test/mean_mean_rank",
    "test/mean_cos",
    "test/mean_f1", 
]


In [2]:
from collections import defaultdict
exp_name_to_key_to_results = defaultdict(lambda: defaultdict(list))
for input_type in ['1d', "all_info", "HSQC_and_C", "HSQC_and_H", "only_C", "only_H", "only_HSQC"]:
# for input_type in ['all_info']:    
    table_path = f'/root/gurusmart/MorganFP_prediction/reproduce_previous_works/puuting_h_in_the_middle/entropy_radius_exps_{input_type}'


    all_exps = os.listdir(table_path)
    # load pickles 
    for exp in all_exps:
        if 'trial' in exp:
            trial_spelling = "trial"
        elif 'trail' in exp:
            trial_spelling = "trail"
           
        try:     
            # print(exp)
            with open(f'{table_path}/{exp}/test_result.pkl', 'rb') as f:
                result = pickle.load(f)[0]
                
                for key in print_keys:
                    if key not in result:
                        continue
                    exp_name_to_key_to_results["_".join(exp.split(trial_spelling)[:-1])][key].append(result[key])
                    # exp_name_to_key_to_results[exp][key].append(result[key])
                    # print(key, result[key])
        except Exception as e:
            print(f'{table_path}/{exp}/test_result.pkl')            
    
exp_name_to_key_to_results


defaultdict(<function __main__.<lambda>()>,
            {'R0_to_R13_only_1d_': defaultdict(list,
                         {'test/mean_rank_1': [0.9064127802848816,
                           0.895751953125,
                           0.9049479365348816],
                          'test/mean_rank_5': [0.9754231572151184,
                           0.9724934697151184,
                           0.9776204228401184],
                          'test/mean_mean_rank': [1.87060546875,
                           2.2333984375,
                           1.4227701425552368],
                          'test/mean_cos': [0.8522032499313354,
                           0.8538790345191956,
                           0.8551177978515625],
                          'test/mean_f1': [0.8629158139228821,
                           0.8648636937141418,
                           0.8650999069213867]}),
             'R0_to_R5_only_1d_': defaultdict(list,
                         {'test/mean_rank_1': [0.917154967

In [3]:
# # compute the std  of rank_1 and rank_5 for each nmr
# for exp_name, key_to_results in exp_name_to_key_to_results.items():
#     for key, results in key_to_results.items():
#         if key in ["test/mean_rank_1", "test/mean_rank_5"]:
#             print(exp_name, key, np.mean(results), np.std(results))
#         # print(exp_name, key, np.mean(results), np.std(results))
#     print()

In [4]:
avg_results = {}
for exp_name, key_to_results in exp_name_to_key_to_results.items():
    avg_results[exp_name] = {key: np.mean(val) for key, val in key_to_results.items()}

In [5]:
# sort based on dict key 
def get_sort_key(item):
    if ((item[0].split("R0_to_R")[-1]).split("_")[0]).isdecimal():
        return int((item[0].split("R0_to_R")[-1]).split("_")[0])
    
    else:
        return float('inf')



avg_results = dict(sorted(avg_results.items(),key=get_sort_key ))

In [6]:
avg_results

{'R0_to_R1_only_1d_': {'test/mean_rank_1': np.float64(0.8830295205116272),
  'test/mean_rank_5': np.float64(0.9671223759651184),
  'test/mean_mean_rank': np.float64(4.317627032597859),
  'test/mean_cos': np.float64(0.9299830396970113),
  'test/mean_f1': np.float64(0.9333892067273458)},
 'R0_to_R1_all_info_': {'test/mean_rank_1': np.float64(0.8975694378217062),
  'test/mean_rank_5': np.float64(0.9751518964767456),
  'test/mean_mean_rank': np.float64(3.835150877634684),
  'test/mean_cos': np.float64(0.9352299769719442),
  'test/mean_f1': np.float64(0.9375181595484415)},
 'R0_to_R1_HSQC_and_C_': {'test/mean_rank_1': np.float64(0.8904622197151184),
  'test/mean_rank_5': np.float64(0.970703125),
  'test/mean_mean_rank': np.float64(5.182345867156982),
  'test/mean_cos': np.float64(0.932726780573527),
  'test/mean_f1': np.float64(0.9354084332784017)},
 'R0_to_R1_HSQC_and_H_': {'test/mean_rank_1': np.float64(0.8141276240348816),
  'test/mean_rank_5': np.float64(0.9400499065717062),
  'test/mea

In [7]:
name_convert = {
    "all_info": "All 3 NMR",
    "1d": "C NMR and H NMR",
    "HSQC_and_C": "HSQC and C NMR",
    "HSQC_and_H": "HSQC and H NMR",
    "only_c": "Only C NMR",
    "only_H": "Only H NMR",
    "only_hsqc": "Only HSQC NMR",
}

# for each combination, find which fp is best
for combination in ["all_info", '1d', "HSQC_and_C", "HSQC_and_H", "only_c", "only_H", "only_hsqc"]:
    print(name_convert[combination], end="")
    for mfp_type in ["R0_to_R1_", "R0_to_R2", "R0_to_R3", "R0_to_R4", "R0_to_R5", "Hyun_FP"]:
        for key, metric_dict in avg_results.items():
            # print(key)
            if combination not in key or mfp_type not in key:
                continue
            for metric, val in metric_dict.items():
                if metric in ["test/mean_rank_1", ]:
                    print(f' & {val*100:.2f}\%', end="")
    print(" \\\\")
    print("\hline")
    

All 3 NMR & 89.76\% & 92.89\% & 93.08\% & 92.93\% & 93.01\% & 91.10\% \\
\hline
C NMR and H NMR & 88.30\% & 91.75\% & 91.66\% & 91.50\% & 91.51\% & 90.00\% \\
\hline
HSQC and C NMR & 89.05\% & 92.13\% & 92.59\% & 92.34\% & 92.35\% & 90.68\% \\
\hline
HSQC and H NMR & 81.41\% & 85.66\% & 85.81\% & 85.53\% & 85.50\% & 83.67\% \\
\hline
Only C NMR & 84.76\% & 88.56\% & 89.19\% & 88.82\% & 88.80\% & 87.74\% \\
\hline
Only H NMR & 60.35\% & 66.00\% & 67.16\% & 66.73\% & 66.20\% & 64.67\% \\
\hline
Only HSQC NMR & 78.99\% & 83.84\% & 83.91\% & 84.03\% & 82.73\% & 81.59\% \\
\hline


In [15]:
for input_type in ["all_info", '1d', "HSQC_and_C", "HSQC_and_H", "only_c", "only_H", "only_hsqc"]:
    print("\n\n")
    # print in latex format
    print ("\\begin{table}[h]")
    print("\centering")
    print('\\begin{tabular}{|c|c|c|c|c|c|}')
    print("\hline")
    print("Model Input & Rank-1$\\uparrow$ & Rank-5$\\uparrow$ & Mean Rank$\\downarrow$ & Cosine Sim$\\uparrow$ & F1-score$\\uparrow$\\\\")
    print("\hline")

    for exp_name, key_to_results in avg_results.items():
        if input_type not in exp_name:
            continue
        
        name = exp_name
        if name[:2]=="FP":
            name = "R"+name.split(" ")[-1][-2]+" FP"
        MF_name = " ".join(name.split("_")[:3]) + " MF"
        if MF_name.startswith("Hyun"):
            MF_name = "DeepSAT FP"
        print(MF_name, end="")
        for key, val in key_to_results.items():
            if key in ["test/mean_rank_1", "test/mean_rank_5"]:
                print(f' & {val*100:.2f}\%', end="")
            elif key in ["test/mean_mean_rank"]:
                print(f' & {val+1:.2f}', end="")
            else:
                print(f' & {val:.4f}', end="")
        print(" \\\\")
        print("\hline")
        
    print('\end{tabular}')
    print("\caption{Performance of various MFs when "+ name_convert[input_type] +" is available}")
    print('\label{table_name}')
    print('\end{table}')




\begin{table}[h]
\centering
\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model Input & Rank-1$\uparrow$ & Rank-5$\uparrow$ & Mean Rank$\downarrow$ & Cosine Sim$\uparrow$ & F1-score$\uparrow$\\
\hline
R0 to R1 MF & 89.76\% & 97.52\% & 4.84 & 0.9352 & 0.9375 \\
\hline
R0 to R2 MF & 92.89\% & 98.33\% & 3.54 & 0.8824 & 0.8892 \\
\hline
R0 to R3 MF & 93.08\% & 98.55\% & 2.78 & 0.8519 & 0.8617 \\
\hline
R0 to R4 MF & 92.93\% & 98.66\% & 3.11 & 0.8437 & 0.8535 \\
\hline
R0 to R5 MF & 93.01\% & 98.77\% & 2.81 & 0.8384 & 0.8483 \\
\hline
R0 to R6 MF & 92.44\% & 98.43\% & 2.48 & 0.8419 & 0.8511 \\
\hline
R0 to R7 MF & 92.31\% & 98.52\% & 3.00 & 0.8445 & 0.8535 \\
\hline
R0 to R8 MF & 92.00\% & 98.46\% & 2.70 & 0.8472 & 0.8563 \\
\hline
R0 to R9 MF & 91.72\% & 98.32\% & 2.85 & 0.8478 & 0.8569 \\
\hline
R0 to R10 MF & 91.70\% & 98.38\% & 2.44 & 0.8523 & 0.8608 \\
\hline
R0 to R11 MF & 91.49\% & 98.48\% & 2.86 & 0.8537 & 0.8625 \\
\hline
R0 to R12 MF & 91.70\% & 98.25\% & 3.24 & 0.8525 & 0.8618 \\
\hli

In [18]:
exp_name

'Hyun_FP_only_hsqc_'

not avg here
just looking at multi vs single clsifer

In [19]:
import os, pickle, sys, torch, numpy as np
from collections import defaultdict


# paths  
table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes'

metric = [
    "mean_rank_1",
    # "mean_rank_5",
    # "mean_mean_rank",
    "mean_cos",
    # "mean_f1", 
]

NMRs  = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc" , "only_1d","only_C_NMR", "only_H_NMR"]

all_keys = [ "test_"+m+"_"+nmr for nmr in NMRs for m in metric]


In [20]:
all_keys

['test_mean_rank_1_all_inputs',
 'test_mean_cos_all_inputs',
 'test_mean_rank_1_HSQC_H_NMR',
 'test_mean_cos_HSQC_H_NMR',
 'test_mean_rank_1_HSQC_C_NMR',
 'test_mean_cos_HSQC_C_NMR',
 'test_mean_rank_1_only_hsqc',
 'test_mean_cos_only_hsqc',
 'test_mean_rank_1_only_1d',
 'test_mean_cos_only_1d',
 'test_mean_rank_1_only_C_NMR',
 'test_mean_cos_only_C_NMR',
 'test_mean_rank_1_only_H_NMR',
 'test_mean_cos_only_H_NMR']

In [21]:

# load pickles 
from collections import defaultdict




In [22]:
def show_result(filter_by="multi"):
    
    exp_name_to_key_to_results = defaultdict(lambda: defaultdict(list))
    all_exps = os.listdir(table_path)
    all_exps = list(filter(lambda x: filter_by in x, all_exps))

    for exp in all_exps:
        # print(exp)
        with open(f'{table_path}/{exp}/test_result.pkl', 'rb') as f:
            result = pickle.load(f)[0]
            # print(result)
            # break
            for key in all_keys:
                if key not in result:
                    continue
                exp_name_to_key_to_results["_".join(exp.split("_")[:-1])][key].append(result[key])
                # print(key, result[key])

    avg_results = {}
    for exp_name, key_to_results in exp_name_to_key_to_results.items():
        avg_results[exp_name] = {key: np.mean(val) for key, val in key_to_results.items()}
    avg_results = dict(sorted(avg_results.items()))
    display(avg_results)
    # for exp_name, key_to_results in avg_results.items():
    #     name = exp_name
    
    #     print(f'{name.replace("_", " ")}', end="")
    #     for key, val in key_to_results.items():
    #         if key in ["test/mean_rank_1", "test/mean_rank_5"]:
    #             print(f' & {val*100:.2f}\%', end="")
    #         elif key in ["test/mean_mean_rank"]:
    #             print(f' & {val:.2f}', end="")
    #         else:
    #             print(f' & {val:.4f}', end="")
    #     print(" \\\\")
    #     print("\hline")


show_result()          

{'flexible_384_multi': {'test_mean_rank_1_all_inputs': np.float64(0.9276204903920492),
  'test_mean_cos_all_inputs': np.float64(0.8507891893386841),
  'test_mean_rank_1_HSQC_H_NMR': np.float64(0.8900325496991476),
  'test_mean_cos_HSQC_H_NMR': np.float64(0.8254165848096212),
  'test_mean_rank_1_HSQC_C_NMR': np.float64(0.9147602717081705),
  'test_mean_cos_HSQC_C_NMR': np.float64(0.8390627900759379),
  'test_mean_rank_1_only_hsqc': np.float64(0.7748593688011169),
  'test_mean_cos_only_hsqc': np.float64(0.7614926695823669),
  'test_mean_rank_1_only_1d': np.float64(0.9058181444803873),
  'test_mean_cos_only_1d': np.float64(0.8369354208310446),
  'test_mean_rank_1_only_C_NMR': np.float64(0.8745523691177368),
  'test_mean_cos_only_C_NMR': np.float64(0.8169729113578796),
  'test_mean_rank_1_only_H_NMR': np.float64(0.7736481428146362),
  'test_mean_cos_only_H_NMR': np.float64(0.777970532576243)}}

In [23]:
show_result("flexible_384_t")          

{'flexible_384': {'test_mean_rank_1_all_inputs': np.float64(0.9239176114400228),
  'test_mean_cos_all_inputs': np.float64(0.849866251150767),
  'test_mean_rank_1_HSQC_H_NMR': np.float64(0.8838695685068766),
  'test_mean_cos_HSQC_H_NMR': np.float64(0.8238423665364584),
  'test_mean_rank_1_HSQC_C_NMR': np.float64(0.9105113744735718),
  'test_mean_cos_HSQC_C_NMR': np.float64(0.8393414616584778),
  'test_mean_rank_1_only_hsqc': np.float64(0.7970621983210245),
  'test_mean_cos_only_hsqc': np.float64(0.7732182741165161),
  'test_mean_rank_1_only_1d': np.float64(0.8979419867197672),
  'test_mean_cos_only_1d': np.float64(0.8331032395362854),
  'test_mean_rank_1_only_C_NMR': np.float64(0.8609496156374613),
  'test_mean_cos_only_C_NMR': np.float64(0.8099772930145264),
  'test_mean_rank_1_only_H_NMR': np.float64(0.7571683526039124),
  'test_mean_cos_only_H_NMR': np.float64(0.7691749930381775)}}

In [24]:
nmr_to_metric_to_value_single = defaultdict(dict)


for i, (k,v) in enumerate(single_dict.items()):
    
    nmr_type = NMR_types[i//14]
    # nmr_type = " ".join(nmr_type.split("_")[2:])
    metric = i%14
    if metric in [4,6, 10, 11, 12]:
        nmr_to_metric_to_value_single[nmr_type][k] = v
    else:
        continue


NameError: name 'single_dict' is not defined

In [ ]:
# nmr_to_metric_to_value_multi = defaultdict(dict)


# for i, (k,v) in enumerate(multi_dict.items()):
    
#     nmr_type = NMR_types[i//14]
#     nmr_type = " ".join(nmr_type.split("_")[2:])
#     metric = i%14
#     if metric in [4,6, 10, 11, 12]:
#         nmr_to_metric_to_value_multi[nmr_type][k] = v
#     else:
#         continue


In [ ]:
# nmr_to_metric_to_value_single

defaultdict(dict,
            {'all_inputs': {'test_mean_cos_all_inputs': 0.8479413390159607,
              'test_mean_f1_all_inputs': 0.8588761687278748,
              'test_mean_mean_rank_all_inputs': 2.579009771347046,
              'test_mean_rank_1_all_inputs': 0.5099216103553772,
              'test_mean_rank_5_all_inputs': 0.9658825397491455},
             'HSQC_H_NMR': {'test_mean_cos_HSQC_H_NMR': 0.8220083713531494,
              'test_mean_f1_HSQC_H_NMR': 0.8360926508903503,
              'test_mean_mean_rank_HSQC_H_NMR': 7.510948657989502,
              'test_mean_rank_1_HSQC_H_NMR': 0.482708215713501,
              'test_mean_rank_5_HSQC_H_NMR': 0.9477534294128418},
             'HSQC_C_NMR': {'test_mean_cos_HSQC_C_NMR': 0.837510883808136,
              'test_mean_f1_HSQC_C_NMR': 0.8496221899986267,
              'test_mean_mean_rank_HSQC_C_NMR': 3.8844082355499268,
              'test_mean_rank_1_HSQC_C_NMR': 0.5028753876686096,
              'test_mean_rank_5_HSQC_C_NMR':

In [ ]:
# latex table
for k,v in nmr_to_metric_to_value_single.items():
    print(k.replace("_", " ")+ " single classifier")
    for print_k in print_keys:
        for metric, val in v.items():
            if print_k in metric:
                if print_k in ["mean_rank_1", "mean_rank_5"]:
                    print(f' & {val*100:.2f}\%', end="")
                elif print_k in ["mean_mean_rank"]:
                    print(f' & {val:.2f}', end="")
                else:
                    print(f' & {val:.4f}', end="")
    print("\\hline")

    print(k.replace("_", " ")+" multi classifier")
    v_multi = nmr_to_metric_to_value_multi[k]
    for print_k in print_keys:                
        for metric, val in v_multi.items():
            if print_k in metric:
                if print_k in ["mean_rank_1", "mean_rank_5"]:
                    print(f' & {val*100:.2f}\%', end="")
                elif print_k in ["mean_mean_rank"]:
                    print(f' & {val:.2f}', end="")
                else:
                    print(f' & {val:.4f}', end="")
    print("\\hline")
   
    

all inputs single classifier
 & 50.99\% & 96.59\% & 2.58 & 0.8479 & 0.8589\hline
all inputs multi classifier
\hline
HSQC H NMR single classifier
 & 48.27\% & 94.78\% & 7.51 & 0.8220 & 0.8361\hline
HSQC H NMR multi classifier
\hline
HSQC C NMR single classifier
 & 50.29\% & 96.48\% & 3.88 & 0.8375 & 0.8496\hline
HSQC C NMR multi classifier
\hline
only hsqc single classifier
 & 44.24\% & 89.46\% & 16.04 & 0.7785 & 0.7949\hline
only hsqc multi classifier
\hline
only 1d single classifier
 & 48.54\% & 94.75\% & 6.19 & 0.8278 & 0.8409\hline
only 1d multi classifier
\hline
only H NMR single classifier
 & 37.68\% & 84.94\% & 36.97 & 0.7602 & 0.7784\hline
only H NMR multi classifier
\hline
only C NMR single classifier
 & 46.34\% & 92.17\% & 14.63 & 0.8033 & 0.8191\hline
only C NMR multi classifier
\hline


In [ ]:
nmr_to_metric_to_value_multi

defaultdict(dict,
            {'': {'test_mean_cos_all_inputs': 0.8487057685852051,
              'test_mean_f1_all_inputs': 0.858889639377594,
              'test_mean_mean_rank_all_inputs': 2.728726863861084,
              'test_mean_rank_1_all_inputs': 0.5132314562797546,
              'test_mean_rank_5_all_inputs': 0.9662854075431824,
              'test_mean_cos_only_hsqc': 0.7571645975112915,
              'test_mean_f1_only_hsqc': 0.7718337774276733,
              'test_mean_mean_rank_only_hsqc': 16.730077743530273,
              'test_mean_rank_1_only_hsqc': 0.4196631610393524,
              'test_mean_rank_5_only_hsqc': 0.8786495327949524,
              'test_mean_cos_only_1d': 0.8341397047042847,
              'test_mean_f1_only_1d': 0.8465748429298401,
              'test_mean_mean_rank_only_1d': 5.494770526885986,
              'test_mean_rank_1_only_1d': 0.4977881610393524,
              'test_mean_rank_5_only_1d': 0.9539307355880737},
             'NMR': {'test_mean_cos_H

In [6]:
import pickle
for split in ['train','val','test']:
    path = f"/root/gurusmart/MorganFP_prediction/reproduce_previous_works/smart4.5/datasets/{split}_indices_of_full_info_NMRs.pkl"
    indices = pickle.load(open(path, 'rb'))
    print(split, len(indices))

train 31990
val 3986
test 3930
